In [20]:
import dataloader.path
import os
import re

from bs4 import BeautifulSoup
import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.data.path.append(dataloader.path.NLTK_DATA)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV


train = pd.read_csv(os.path.join(dataloader.path.IMDB, 'labeledTrainData.tsv'), delimiter='\t')
test = pd.read_csv(os.path.join(dataloader.path.IMDB, 'testData.tsv'), delimiter='\t')

AttributeError: module 'dataloader.path' has no attribute 'NLTK_DATA'

In [12]:
train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [16]:
def review_to_text(review, remove_stopwords):
    
    # 去除html标签
    raw_text = BeautifulSoup(review, 'html').get_text()
    
    # 去除非字母字符
    letters = re.sub('[^a-zA-z]', ' ', raw_text)
    words = letters.lower().split()
    
    # 去除停用词
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if w not in stop_words]
    
    # 返回处理完的列表
    return words

In [18]:
x_train = list()
for review in train['review']:
    x_train.append(' '.join(review_to_text(review, True)))

x_test = list()
for review in test['review']:
    x_test.append(' '.join(review_to_text(review, True)))
    
y_train = train['sentiment']

pip_count = Pipeline([('count_vec', CountVectorizer(analyzer='word')), ('mnb', MultinomialNB())])
pip_tfidf = Pipeline([('tfidf_vec', TfidfVectorizer(analyzer='word')), ('mnb', MultinomialNB())])

params_count = {'count_vec_binary': [True, False], 
                'count_vec_ngram_range': [(1, 1), (1, 2)],
                'mnb_alpha': [0.1, 0.1, 10.0]}

params_tfidf = {'tfidf_vec_binary': [True, False], 
                'tfidf_vec_ngram_range': [(1, 1), (1, 2)],
                'mnb_alpha': [0.1, 0.1, 10.0]}

gs_count = GridSearchCV(pip_count, params_count, cv=4, n_jobs=-1, verbose=1)
gs_count.fit(x_train, y_train)

print(gs_count.best_params_)
print(gs_count.best_score_)

/Users/nemos/anaconda/envs/ml/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/nemos/anaconda/envs/ml/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


LookupError: 
**********************************************************************
  Resource 'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/Users/nemos/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************